In [10]:
import rqdatac
import datetime,calendar

In [ ]:
# 设置回测信息
start_date = datetime.date(2021,1,1)
end_date = datetime.date(2024,12,1)
tradingdays_list = rqdatac.get_trading_dates(start_date,end_date)

In [ ]:
# 调仓日：每年6月和12月的第二个星期五的下一交易日
rebalancing_days = [start_date]
# 生成start_date和end_date之间的所有年份列表
years = list(range(start_date.year, end_date.year + 1))
# 生成years中每个年份6月和12月的第二个星期五的下一个交易日
for year in years:
    for month in [6, 12]:
        date = datetime.date(year, month, 1)
        while date.month == month:
            if date.weekday() == 4: # 星期五
                second_friday = date + datetime.timedelta(days=7) # 下一个星期五
                # 找到tradingdays_list中第一个大于等于second_friday的日期
                while second_friday not in tradingdays_list and second_friday <= end_date:
                    print(second_friday)
                    second_friday += datetime.timedelta(days=1)
                rebalancing_days.append(second_friday) # 添加调仓日
                break # 找到第二个星期五之后的下一个交易日，跳出循环
            else:
                date += datetime.timedelta(days=1) # 下一天

In [40]:
# 在每一个rebalancing_days中的调仓日，用rqdatac获取最新的中证1000指数样本空间
# for i in range(len(rebalancing_days)):

tradingday_new = tradingdays_list[0]
# 用rqdatac获取最新的中证1000指数样本空间
sample_space = rqdatac.index_components('000852.XSHG', tradingday_new)


<class 'list'>


In [46]:
# 在样本空间sample_space中，选取tradingday_new之前连续三年有现金分红的股票
dividend_info = rqdatac.get_dividend_info(
    sample_space,  tradingday_new - datetime.timedelta(days=365*3),tradingday_new)

info_date ex_dividend_date dividend_type  \
order_book_id effective_date                                              
000006.XSHE   2018-06-30      2018-08-17             None   no dividend   
              2018-12-31      2019-06-12       2019-06-19     cash only   
              2019-06-30      2019-08-16             None   no dividend   
              2019-12-31      2020-06-12       2020-06-18     cash only   
              2020-06-30      2020-08-28             None   no dividend   
...                                  ...              ...           ...   
688366.XSHG   2020-06-30      2020-08-28             None   no dividend   
              2020-12-31      2021-07-22       2021-07-29     cash only   
688388.XSHG   2019-12-31      2020-05-14       2020-05-20     cash only   
              2020-06-30      2020-08-20             None   no dividend   
              2020-12-31      2021-04-27       2021-05-06     cash only   

                              dividend_bonus_share  \
order_book_id effective_date                         
000006.XSHE   2018-06-30                       NaN   
              2018-12-31                       NaN   
              2019-06-30                       NaN   
              2019-12-31                       NaN   
              2020-06-30                       NaN   
...                                            ...   
688366.XSHG   2020-06-30                       NaN   
              2020-12-31                       NaN   
688388.XSHG   2019-12-31                       NaN   
              2020-06-30                       NaN   
              2020-12-31                       NaN   

                              dividend_transferred_share  \
order_book_id effective_date                               
000006.XSHE   2018-06-30                             NaN   
              2018-12-31                             NaN   
              2019-06-30                             NaN   
              2019-12-31                             NaN   
              2020-06-30                             NaN   
...                                                  ...   
688366.XSHG   2020-06-30                             NaN   
              2020-12-31                             NaN   
688388.XSHG   2019-12-31                             NaN   
              2020-06-30                             NaN   
              2020-12-31                             NaN   

                              dividend_cash_before_tax  
order_book_id effective_date                            
000006.XSHE   2018-06-30                           NaN  
              2018-12-31                          1.95  
              2019-06-30                           NaN  
              2019-12-31                          1.79  
              2020-06-30                           NaN  
...                                                ...  
688366.XSHG   2020-06-30                           NaN  
              2020-12-31                          5.00  
688388.XSHG   2019-12-31                          4.30  
              2020-06-30                           NaN  
              2020-12-31                          2.50  

[5710 rows x 6 columns]

In [ ]:
# 根据dividend_info筛选出连续三年有现金分红的股票
